In [1]:
import os
import openai
import warnings
warnings.filterwarnings('ignore')

from dotenv import load_dotenv
load_dotenv()
openai.api_key = os.environ['OPENAI_API_KEY']

import sys
sys.path.append("..")
from utils_Metrics import get_prebuilt_trulens_recorder

🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `Tru` to prevent this.


In [2]:
from llama_index import SimpleDirectoryReader

# Loading and cchunking the documents

documents=SimpleDirectoryReader(
    input_files=["../data/Machine Learning Engineering with Python-2023.pdf"]
).load_data()


In [3]:
print(type(documents), "\n")
print(len(documents), "\n")
print(type(documents[0]))
print(documents[0])

<class 'list'> 

463 

<class 'llama_index.schema.Document'>
Doc ID: 061c4a11-430b-4244-800c-ed94cdc44af3
Text:


In [4]:
from llama_index import Document

document = Document(text="\n\n".join([doc.text for doc in documents]))

### Window-sentence retrieval setup


In [5]:
from llama_index.node_parser import SentenceWindowNodeParser

# create the sentence window node parser w/ default settings
node_parser = SentenceWindowNodeParser.from_defaults(
    window_size=3,
    window_metadata_key="window",
    original_text_metadata_key="original_text",
)

In [6]:
text = "hello. how are you? I am fine!  "

nodes = node_parser.get_nodes_from_documents([Document(text=text)])

In [7]:
print([x.text for x in nodes])

['hello. ', 'how are you? ', 'I am fine!  ']


In [8]:
print(nodes[1].metadata["window"])

hello.  how are you?  I am fine!  


In [9]:
text = "hello. foo bar. cat dog. mouse"

nodes = node_parser.get_nodes_from_documents([Document(text=text)])

In [10]:
print([x.text for x in nodes])

['hello. ', 'foo bar. ', 'cat dog. ', 'mouse']


In [11]:
print(nodes[0].metadata["window"])

hello.  foo bar.  cat dog. 


### Building the Inedx

In [12]:
from llama_index.llms import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)

In [13]:
from llama_index import ServiceContext
from llama_index.embeddings import HuggingFaceEmbedding


#config embedding
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
sentence_context = ServiceContext.from_defaults(
    llm=llm,
    embed_model=embed_model,
    # embed_model="local:BAAI/bge-large-en-v1.5"
    node_parser=node_parser,
)

In [14]:
from llama_index import VectorStoreIndex

sentence_index = VectorStoreIndex.from_documents(
    [document], service_context=sentence_context
)

In [15]:
sentence_index.storage_context.persist(persist_dir="./sentence_index")


In [16]:
# This block of code is optional to check
# if an index file exist, then it will load it
# if not, it will rebuild it

import os
from llama_index import VectorStoreIndex, StorageContext, load_index_from_storage
from llama_index import load_index_from_storage

if not os.path.exists("./sentence_index"):
    sentence_index = VectorStoreIndex.from_documents(
        [document], service_context=sentence_context
    )

    sentence_index.storage_context.persist(persist_dir="./sentence_index")
else:
    sentence_index = load_index_from_storage(
        StorageContext.from_defaults(persist_dir="./sentence_index"),
        service_context=sentence_context
    )

### ### Building the postprocessor

In [17]:
from llama_index.indices.postprocessor import MetadataReplacementPostProcessor

postproc = MetadataReplacementPostProcessor(
    target_metadata_key="window"
)

In [18]:
from llama_index.schema import NodeWithScore
from copy import deepcopy

scored_nodes = [NodeWithScore(node=x, score=1.0) for x in nodes]
nodes_old = [deepcopy(n) for n in nodes]

In [19]:
nodes_old[1].text

'foo bar. '

In [20]:
replaced_nodes = postproc.postprocess_nodes(scored_nodes)

In [21]:
print(replaced_nodes[1].text)

hello.  foo bar.  cat dog.  mouse


### Adding a reranker

Got the below error : while running below code 

Error:
ValueError: Couldn't instantiate the backend tokenizer from one of: (1) a `tokenizers` library serialization file, (2) a slow tokenizer instance to convert or (3) an equivalent slow tokenizer class to instantiate and convert. You need to have sentencepiece installed to convert a slow tokenizer to a fast one.

Solution:

I resolved it.

Uninstalled transformers
Installed transformers sentencepiece like this : !pip install --no-cache-dir transformers sentencepiece


In [22]:
from llama_index.indices.postprocessor import SentenceTransformerRerank

# BAAI/bge-reranker-base
# link: https://huggingface.co/BAAI/bge-reranker-base
rerank = SentenceTransformerRerank(
    top_n=2, model="BAAI/bge-reranker-base"
)

In [23]:
from llama_index import QueryBundle
from llama_index.schema import TextNode, NodeWithScore

query = QueryBundle("I want a dog.")

scored_nodes = [
    NodeWithScore(node=TextNode(text="This is a cat"), score=0.6),
    NodeWithScore(node=TextNode(text="This is a dog"), score=0.4),
]

In [24]:
reranked_nodes = rerank.postprocess_nodes(
    scored_nodes, query_bundle=query
)

In [25]:
print([(x.text, x.score) for x in reranked_nodes])

[('This is a dog', 0.9182743), ('This is a cat', 0.0014040773)]


### Runing the query engine

In [26]:
sentence_window_engine = sentence_index.as_query_engine(
    similarity_top_k=6, node_postprocessors=[postproc, rerank]
)

In [27]:
window_response = sentence_window_engine.query(
    "How to publish in ECR"
)

In [28]:
from llama_index.response.notebook_utils import display_response

display_response(window_response)

**`Final Response:`** To publish a container in ECR, you need to follow these steps:

1. Create an ECR repository using the AWS CLI command `aws ecr create-repository`. Specify the repository name and the desired region. Make sure to enable image scanning by setting the `--image-scanning-configuration` flag to `scanOnPush=true`.

2. After creating the repository, you will receive metadata that includes the repository URI. This URI will be needed for the next step.

3. Log in to the container registry using the `docker login` command in the Terminal. Use the repository URI provided in the metadata.

4. Build your container image using the Dockerfile defined in your project directory.

5. Tag the built image with the repository URI using the `docker tag` command.

6. Push the tagged image to the ECR repository using the `docker push` command.

By following these steps, you can successfully publish your container in ECR.

## Putting it all Together

In [30]:
import os
from llama_index import ServiceContext, VectorStoreIndex, StorageContext
from llama_index.node_parser import SentenceWindowNodeParser
from llama_index.indices.postprocessor import MetadataReplacementPostProcessor
from llama_index.indices.postprocessor import SentenceTransformerRerank
from llama_index import load_index_from_storage
from llama_index.embeddings import HuggingFaceEmbedding


#config embedding
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")


def build_sentence_window_index(
    documents,
    llm,
    embed_model=embed_model,
    sentence_window_size=3,
    save_dir="sentence_index",
):
    # create the sentence window node parser w/ default settings
    node_parser = SentenceWindowNodeParser.from_defaults(
        window_size=sentence_window_size,
        window_metadata_key="window",
        original_text_metadata_key="original_text",
    )
    sentence_context = ServiceContext.from_defaults(
        llm=llm,
        embed_model=embed_model,
        node_parser=node_parser,
    )
    if not os.path.exists(save_dir):
        sentence_index = VectorStoreIndex.from_documents(
            documents, service_context=sentence_context
        )
        sentence_index.storage_context.persist(persist_dir=save_dir)
    else:
        sentence_index = load_index_from_storage(
            StorageContext.from_defaults(persist_dir=save_dir),
            service_context=sentence_context,
        )

    return sentence_index


def get_sentence_window_query_engine(
    sentence_index, similarity_top_k=6, rerank_top_n=2
):
    # define postprocessors
    postproc = MetadataReplacementPostProcessor(target_metadata_key="window")
    rerank = SentenceTransformerRerank(
        top_n=rerank_top_n, model="BAAI/bge-reranker-base"
    )

    sentence_window_engine = sentence_index.as_query_engine(
        similarity_top_k=similarity_top_k, node_postprocessors=[postproc, rerank]
    )
    return sentence_window_engine

In [31]:
from llama_index.llms import OpenAI

index = build_sentence_window_index(
    [document],
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0.1),
    save_dir="./sentence_index",
)


In [32]:
query_engine = get_sentence_window_query_engine(index, similarity_top_k=6)
